In [2]:
!apt-get -y install poppler-utils
!wget https://github.com/tesseract-ocr/tessdata/raw/main/guj.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/
!pip install -q pdf2image pytesseract deep-translator transformers accelerate bitsandbytes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 84 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 700 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.8 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.8 [5,186 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.8 [1,072 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 1,46

In [3]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import re


def preprocess_image_for_tesseract(pil_img):
    img = np.array(pil_img.convert('L'))  
    img = cv2.GaussianBlur(img, (3, 3), 0)  
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  
    return Image.fromarray(thresh)


pdf_path = "/kaggle/input/dfdfwf/1333.pdf"
pages = convert_from_path(pdf_path, dpi=450)


combined_text = []
for i, page in enumerate(pages):
    preprocessed = preprocess_image_for_tesseract(page)
    text = pytesseract.image_to_string(preprocessed, lang="guj+eng")
    combined_text.append(text)

# Step 4: Join & clean text
raw_text = "\n".join(combined_text)

def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_text = clean_text(raw_text)


print(cleaned_text)  



#print(gujarati_text)  

| સ્પે. પોકસો કેસ નં.રર/ર૦ર૧ આંક: વિરમગામના મહે. આઠમાં એડિ. ડિસ્ટ્રિક્ટ એન્ડ સેશન્સ જજ (એ. એમ. વર્મા) સાહેબશ્રીની કોર્ટમાં, તા.વિરમગામ, જિ.અમદાવાદ. ચુકાદાની તા. ૧૪/૦૫/ર૦રપ પોકસો કેસ નં.રર/ર૦ર૧ (₹.1.[૨.ની વિગત /ગુનો અને પોલીસ સ્ટેશન) વિરમગામ રૂરલ પોલીસ સ્ટેશન પાર્ટ-એ ગુના રજિસ્ટર નં.: ૧૧૧૯ર૦૬૦ર૧૦૦૯૪,/ર૦ર૧ ગુનો: ઈ.પી.કો.ની કલમ- ૩૬૩, ૩૬૬, ૩૭૬(ડો) તથા પોક્સો એક્ટની કલમ-૪, ૮, ૧ર મુજબ પોલીસ સ્ટેશન: વિરમગામ રૂરલ, તા.વિરમગામ, જિ.અમદાવાદ. ફરિયાદી શ્રીસરકાર વિ.મ.સ.વ. મારફતે વિદ્વાન એ.પી. પી.શ્રી એચ. આર. સોલકી (૧) ધનશ્યામભાઈ કાંતીભાઈ ઉર્ફે કનાભાઈ મેર (કો.પટેલ), ઉ.વ.ર૩, આરોપીઓ (ર) રોહિતભાઈ ધીરૂભાઈ કો.પટેલ, ઉં.વ.ર૦, રહે.: નં.૧ કુમણખાણ, તા:વિરમગામ,જી:અમદાવાદ. રહે.: નં.ર ઉડવીગામ, તા: થાનગઢ,જી:સુરેન્દનગર. આરોપીઓ તરફે નં.૧ તરફે વિ. વકીલશ્રી એ. એમ. સોલંકી નં.ર તરફે વિ. વકીલશ્રી જે.એસ.ગળથરા ફોર્મ-બી ગુનો બન્યાની તારીખ ૧૧/૦૧/ર૦ર૧ ફરિયાદની તારીખ ૧૮/૦૩/ર૦ર૧ ચાર્જશીટની તારીખ ૦૪/૧૦/ર૦ર૧ તહોમત ફરમાવ્યાની તારીખ ૧૩/૦૧/ર૦ર૪ 2 સ્પે. પોકસો કેસ નં.રર/ર૦૨ર૧ પુરાવો શરૂ કર્યાની તારીખ ૧૩/૧૧/ર૦ર૪ ચુકાદો સરક્ષિત રાખ્યાની 

In [12]:
import re
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from huggingface_hub import login


HF_TOKEN = ""  


model_name = "ai4bharat/indictrans2-indic-en-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, token=HF_TOKEN, trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


def translate_gujarati_to_english_long(text, chunk_size=450):
    prefix = "guj_Gujr eng_Latn "
    text = re.sub(r'\s+', ' ', text).strip()
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        input_text = prefix + chunk
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_length=512)
        output = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
        translated_chunks.append(output)

    return ' '.join(translated_chunks)


translated_text = translate_gujarati_to_english_long(cleaned_text)
print("\n📘 Full Translated Text:\n", translated_text)



📘 Full Translated Text:
 Criminal Case No.1932 / 1998 Judgment in the Court of Viramgam. Additional Chief Judicial Magistrate ( K.P.Sakor ) in the Court of Sahibshree. For example, the date of hearing of the case of Vidwan A.P.P.Sri M.A.Dhasura.O.P.O ( Jai ) Bhartabhai Amratbhai Sattaru Raheo Amdavad Dariyapur Vadigama Hata Pol.O.P.T. The time frame of the CrPC ofl the claim-strials is the time frame of the trial court in which the trial court decides the fate of the accused in the land dispute.The time frame of the commission is to determine the fate of the accused. The court's bench of Justice Abdul Wahhab Miah and Justice Md Khurshid Alam Khan heard the arguments of the prosecution and the defence in the case. The amount of the housing fee was with AWELL and the buyer and the seller told them that they had hired the salesman. This is the same case where a customer has applied for a Bank of Baroda loan and has been given a Widow's Bank account.The amount of money paid to the benefic

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch


tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-1.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-1.3B")
device = 0 if torch.cuda.is_available() else -1


translator = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="guj_Gujr",  
    tgt_lang="eng_Latn",  
    device=device
)


def translate_gujarati_text(text, max_chunk_len=512):
    inputs = [text[i:i+max_chunk_len] for i in range(0, len(text), max_chunk_len)]
    translations = []
    for chunk in inputs:
        if len(chunk.strip()) < 5:
            continue
        try:
            out = translator(chunk, max_length=512)[0]["translation_text"]
            translations.append(out)
        except Exception as e:
            translations.append(f"[Translation Error]: {e}")
    return " ".join(translations)

english_translation = translate_gujarati_text(cleaned_text)



2025-05-24 12:52:54.975277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748091175.147708      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748091175.201308      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [5]:
print(english_translation)

Spe.Poksho case No.Rr/R0r1 Number: Ma. 8th Ed. District and Sessions Judge (A.M. Verma) in Sahesri Court, Ta.Virmagam, G.A. Ahmedabad Judgment date: 14/05/R0rp Poksho case No.Rr/R0r1 (₹.1.) Details of the case /Gun and Police Station Viramgamural Police Station Part-A Crime Register No: 1119 R0601094 /R0r1 Crimes: Section 363, 366, 376 of the EPCO and Section 4 of the Act, Act No. 8 of the Act, Act No. 4 of the Act, Act No. 8 of the Act, Act No. 4 of the Act, Act No. 1 of the Act, Act No. 4 of the Act, Act No. 4 of the Act, Act No. 8 of the Act, Act No. 4 of the Act, Act No. 1 of the Act, Act No. Yamabhai Kantibhai aka Kanabhai Mer (K.Patel), U.V.R3, accused (s) Rohitbhai Dhirubhai K.Patel, U.V.R0, Rakh: No.1 Kumankhan, Ta: Viramgam, G: Ahmedabad. Rakh: No.1 Udvigam, Ta: Thangarh, G: Surendhangar. Accused (s) to No.1 v. Advocate A.M. Solanki to No. V. Advocate J.S. Galthara-B. Criminal form 11/1/Full Charge Form 11/Full Charge Form 18/Full Charge Form 10/Full Charge Form 10/Full Charge

In [5]:
import re
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration
import textwrap


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SUMMARY_MODEL = "facebook/bart-large-cnn"
EXTRACTION_MODEL = "google/flan-t5-base"

bart_tokenizer = BartTokenizer.from_pretrained(SUMMARY_MODEL)
bart_model = BartForConditionalGeneration.from_pretrained(SUMMARY_MODEL).to(DEVICE)

t5_tokenizer = T5Tokenizer.from_pretrained(EXTRACTION_MODEL)
t5_model = T5ForConditionalGeneration.from_pretrained(EXTRACTION_MODEL).to(DEVICE)


def chunk_text(text, max_tokens=1024):
    return textwrap.wrap(text, width=1024, break_long_words=False)

def summarize_large_text(text):
    chunks = chunk_text(text)
    summaries = []
    for chunk in chunks:
        inputs = bart_tokenizer(chunk, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
        summary_ids = bart_model.generate(inputs["input_ids"], max_length=256, min_length=30, num_beams=4)
        summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return " ".join(summaries)


def extract_fields_t5(text):
    prompt = f"""
Extract the following details from the legal judgment:

- Case Number
- Date of Judgment
- Petitioner / Plaintiff
- Respondent / Defendant

Text:
{text}
"""
    inputs = t5_tokenizer(prompt.strip(), return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
    outputs = t5_model.generate(**inputs, max_new_tokens=256)
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\n🔍 T5 Model Raw Output:\n", response)

    fields = {
        "Case Number": "",
        "Date of Judgment": "",
        "Petitioner / Plaintiff": "",
        "Respondent / Defendant": "",
    }
    for line in response.split("\n"):
        if ":" in line:
            key, val = line.split(":", 1)
            key, val = key.strip(), val.strip()
            if key in fields:
                fields[key] = val
    return fields


def extract_fields_regex(text, current_fields):
    if not current_fields["Case Number"]:
        match = re.search(r"(Criminal\s+)?Case\s+No[:.]?\s*(\S+)", text, re.IGNORECASE)
        if match:
            current_fields["Case Number"] = match.group(2)
    
    if not current_fields["Date of Judgment"]:
        match = re.search(r"Judgment\s+is\s+dated\s+([0-9]{2}/[0-9]{2}/[0-9]{4})", text, re.IGNORECASE)
        if not match:
            match = re.search(r"Date\s+of\s+Judgment[:.]?\s*([0-9]{2}/[0-9]{2}/[0-9]{4})", text, re.IGNORECASE)
        if match:
            current_fields["Date of Judgment"] = match.group(1)

    if not current_fields["Petitioner / Plaintiff"]:
        match = re.search(r"complainant[:,\s]+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)", text)
        if match:
            current_fields["Petitioner / Plaintiff"] = match.group(1)

    if not current_fields["Respondent / Defendant"]:
        match = re.search(r"accused[:,\s]+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)", text)
        if match:
            current_fields["Respondent / Defendant"] = match.group(1)

    return current_fields
def process_document(text):
    fields = extract_fields_t5(text)
    fields = extract_fields_regex(text, fields)
    summary = summarize_large_text(text)
    fields["Summary of Decision"] = summary

    print("\n📄 Final Extracted Fields:\n", fields)
    return fields


def process_documents(texts):
    all_data = []
    for text in texts:
        data = process_document(text)
        all_data.append(data)
    return pd.DataFrame(all_data)



df = process_documents([english_translation])
df.to_csv("legal_summary_output.csv", index=False)
print("✅ CSV saved as 'legal_summary_output.csv'")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


🔍 T5 Model Raw Output:
 - Case Number - Date of Judgment - Petitioner / Plaintiff - Respondent / Defendant Text: Criminal Case No. 1932/1998 Judgment in the court of Mah. Additional Chief Judge Magistrate (K.P. Thakur) Sahasri, Ta.Viramgam, G.Ahmadabad. Judgment is dated 05/03/20025 Case No. 1932/1998 Number: [...]2. Details/Guns and Police Station Ramagam Town Police Station First. Register No. 145/1998. Crime: EPCO Clauses-420, 468, 47 and 511. According to the police station Division Virabhagaon, Police A.P.A. Amravati, G.R. Amravati (government of Gujarat) has been charged with the murder of the accused. The accused is none. Form-B Date of Commitment 28/08/98 Date of appeal 28/08/98 Charge sheet Date 05/12/98 Date of indictment 12/01/2009 Date of commencement of evidence 21/01/2009 Date of preservation of judgment 5/03/2005 Date of such

📄 Final Extracted Fields:
 {'Case Number': '1932/1998', 'Date of Judgment': '05/03/2002', 'Petitioner / Plaintiff': 'Jayeentibhai Govindbhai', 'R